In [1]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output, HTML
import matplotlib.pyplot as plt
from pydrake.all import *
# from manipulation import running_as_notebook
# from manipulation.meshcat_cpp_utils import(
#     StartMeshcat, MeshcatJointSlidersThatPublish)
# from manipulation.scenarios import AddMultibodyTriad

# Import pygmo for optimisation
import pygmo as pg
import pandas as pd

In [6]:
# GLOBAL PROBLEM SETUP

# Reference: https://wiki.ros.org/pr2_controller_manager/safety_limits
# Defining the actuator box
class ActuatorBox(LeafSystem):
    # Input values are for motor and NOT for the joint
    def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
        LeafSystem.__init__(self)
        # Both the values are motor specific NOT joint specific
        # Get joint max torque
        self.tau_max_j = tau_max_m*gear_ratio
        self.k_vel = k_velocity
        # Convert RPM of motor to joint velocity in rad/sec
        self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
        self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
        self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
        self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

    def OutputJointTorque(self, context, output):
        # Get the state values
        q = self.EvalVectorInput(context, 1).get_value()
        qvel = q[na:]
        u = self.EvalVectorInput(context, 0).get_value()
        # Compute the instantaneous torque limit
        # For the AK-70-10 motors
        kv = self.k_vel
        u_new = np.copy(u)
        # Using URDF safety controller rules
        for i in range(len(u)):
            if qvel[i]>=0 and u[i]>=0:
                u_bound = -kv*(qvel[i]-self.omega_max_j)
                u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
            if qvel[i]>=0 and u[i]<=0:
                u_bound = -self.tau_max_j
                u_new[i] = np.max([u[i], u_bound])
            if qvel[i]<=0 and u[i]>=0:
                u_bound = self.tau_max_j
                u_new[i] = np.min([u[i], u_bound])
            if qvel[i]<=0 and u[i]<=0:
#                 u_bound = -kv*(qvel[i]+self.omega_max_j)
                u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
#             print(qvel[i], u[i], u_bound, u_new, '\n')
        output.SetFromVector(u_new)

# Define the number of actuators
na = 7

# Task definition
# Home position
q0 = np.array([3.04, 0.07, 2.47, -0.48, 2.06, -0.89, 0.62])
# Pre defined joint locations as long as design isnt changed
qi = np.array([-2.44638919,0.23876803,1.59048134,0.09794963,3.00620177,-0.84260006,-1.43429923])
qint = np.array([-2.38357708,0.08599855,2.14263713,-0.47696921,2.73985728,-1.57634596,-1.86168229])
qd = np.array([-2.53725333,-0.32203878,1.97857906,-0.12785049,3.03252099,-0.9631777,-2.07610007])


# Pick location
carti = np.array([-0.33, 0.07, 0.8])
# Enforcing the end-effector to be parallel with the table
orii = np.array([-1.57, 0, 0])
Ri = RotationMatrix(RollPitchYaw(orii))
# Place location
cartd = np.array([-0.03, 0.07, 0.8])
# Intermediate location
cartint = (cartd+carti)/2+np.array([0,-0.05,0.2])

# Simulation settings
time_step = 1e-4
sim_time = 5
error_threshold = 5e-3

# Define design variables here
# Assumed values
tau_mu = 5.3e-3
b0 = 1.5e-5
# AK70-10 parameters
# gear ratio
gear_ratio = 10
# motor design variables
tau_max_m = 12/gear_ratio # Nm
omega_max_m = 475*gear_ratio   # RPM
Kt = 0.095        # Nm/A
Kv = 100          # RPM/V
Km = 0.19         # Nm/sqrt(W)
# Computed matrix for joule loss
K_joule = 1/np.sqrt(Km)*np.identity(na)
k_velocity = 8.33/(75*2*np.pi/60) # Nm/(rad/s)

# Control attributes
Kp = 15

# case = "baseline_control"
case = "baseline_motor"

if case=="baseline_control":
    # For baseline-control study
    pre_val_vals = [tau_mu, b0, Km, Kt, Kv, k_velocity, tau_max_m, omega_max_m, gear_ratio]
    pre_val_names = ["tau_mu", "b0", "Km", "Kt", "Kv", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio"]
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["Kp", "Kd"]
    print(pre_val, dv_names)
    
if case=="baseline_motor":
    dv_names = ["Kp", "Kd", "tau_mu", "b0", "Km", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio"]
    print(dv_names)

['Kp', 'Kd', 'tau_mu', 'b0', 'Km', 'k_velocity', 'tau_max_m', 'omega_max_m', 'gear_ratio']


In [7]:
def compute_QoI(pre_val, dv_names, sample_dv):
    # Construct the master dv list
    dv = {**pre_val, **dict(zip(dv_names, sample_dv))}
    # Enter the robot path and finalise sim settings
    K_joule = 1/np.sqrt(dv['Km'])*np.identity(na)
    urdf_path = './urdfs/DIVAR2V/urdf/DIVAR2V_eef.urdf'
    # Construct the sim diagram
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    # Due to the way the problem is formulated, spheres and the table can be safely ignored
    plant.Finalize()
    actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    
    # Initialise controller params, for now limited to Kp and Kd search
    Kp = np.full(na, dv['Kp'])
    Ki = np.full(na, 0)
    Kd = np.full(na, dv['Kd'])
    
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    # Complete connections
    builder.Connect(plant.get_state_output_port(model),
                iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(), 
                actuator.get_input_port(0))
    builder.Connect(plant.get_state_output_port(model),
                actuator.get_input_port(1))
    builder.Connect(actuator.get_output_port(), plant.get_actuation_input_port())
    
    # Connecting a data logger
    # Commanded torque
    logger1 = LogVectorOutput(iiwa_controller.get_output_port_control(), builder)
    # Joint state
    logger2 = LogVectorOutput(plant.get_state_output_port(model), builder)
    # Realised torque
    logger3 = LogVectorOutput(actuator.get_output_port(), builder)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper_frame = plant.GetBodyByName("eef").body_frame()
    
    # Now computing the QoIs
    # WORKSPACE
    # Workspace is always satisfied since its the same robot
    workspace = 0
    # DEFLECTION
    # Deflection is not applicable as we consider only rigid components
    deflection = 0
    # APPLIED FORCE
    # Applied force will also remain the same given that the tau_max is fixed
    applied_force = 0
    # PAYLOAD
    # same as applied force
    payload = 0
    # Setup the simulation
    xd = np.hstack((qint, 0*qint))
    plant.SetPositions(plant_context, qi)
    iiwa_controller.GetInputPort('desired_state').FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), xd);
    # Simulation
    simulator = Simulator(diagram, context)
    simulator.AdvanceTo(sim_time);
    
    # Now we want to get the simulation data, i.e., 
    # the torque applied, final time of the joint variables
    log1 = logger1.FindLog(context)
    log2 = logger2.FindLog(context)
    log3 = logger3.FindLog(context)
    
    error = np.abs(log2.data()[:na,:].transpose() - np.tile(qint,(log3.data().shape[1],1)))
    e_norm = np.array([np.linalg.norm(val) for val in error])
    for i in range(len(e_norm)):
        if np.all(e_norm[i:]<error_threshold):
            break
    final_error = e_norm[-1]
    # CYCLE TIME
    ts = log2.sample_times()[i]
    
    omega_j = []
    for i in range(log2.data().shape[1]):
        omega_j.append(log2.data()[:,i][na:])
    omega_j = np.array(omega_j).transpose()
    omega_m = omega_j*gear_ratio
    # Convert the motor speed to RPM
    omega_m_rpm = omega_m*60/(2*np.pi)
    # Motor torque realised
    tau_m = log3.data()/gear_ratio
    
    # Total power losses
    inst_friction_power = []
    inst_joule_power = []
    for i in range(len(log3.sample_times())):
        omega_inst = omega_m[:, i]
        tau_f = dv['tau_mu']*np.sign(omega_inst)+dv['b0']*omega_inst
        P_f = np.dot(tau_f, omega_inst)
        inst_friction_power.append(P_f)
        tau_t = tau_f+tau_m[:, i]
        P_t = np.dot(K_joule.dot(tau_t), tau_t)
        inst_joule_power.append(P_t)
    
    # POWER LOSSES
    total_friction_loss = np.sum(inst_friction_power)*time_step
    total_joule_loss = np.sum(inst_joule_power)*time_step
    
    results = np.array([workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error])
    print(results)
    return results

In [8]:
# test with a sample
# case-1:['Kp', 'Kd']
# sample_dv = [15, 2*np.sqrt(15)]
# case-2: ['Kp', 'Kd', 'tau_mu', 'b0', 'Km', 'k_velocity', 'tau_max_m', 'omega_max_m', 'gear_ratio']
sample_dv = [15, 2*np.sqrt(15), 0.0053, 1.5e-5, 0.19, 1.0606, 1.2, 4750, 10]
compute_QoI(pre_val, dv_names, sample_dv)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.97380000e+00 1.48929578e-01 1.40334040e+01 9.73874246e-08]


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.97380000e+00, 1.48929578e-01, 1.40334040e+01, 9.73874246e-08])

In [10]:
# Setting up a problem
class Baseline_Control:
    def __init__(self):
        pass
    
    def fitness(self,sample_dv):
        print(sample_dv)
        # Write a function to output the QoI values
        results = compute_QoI(pre_val, dv_names, sample_dv)
        # Add weighting factor, K_w
        # workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error
        K_w = np.diag([1,1,1,1,1,1,1,1])
        # Quadratic objective meta function with weights
        fitness_val = np.matmul(K_w, results).dot(results)
        fitness_val = results.sum()
        return [fitness_val]
    
    def get_bounds(self):
        # Define bounds on the controller variables
        # Ideally from a csv file like before so that one can easily extend the optimisation problem
        if case=="baseline_control":
            dv_range = ([0,0],[100,100])
        if case=="baseline_motor":
            dv_range = ([0,0,0,0,0,0,0,0,0],[20,20,0.01,5e-3,1,2,3,6000,15])
        return dv_range
    
    def get_name(self):
        # get the name of the problem also from the csv sheet
        return "Baseline_Control"

In [11]:
prob = pg.problem(Baseline_Control())
print(prob)
print(prob.get_bounds())

Problem name: Baseline_Control
	C++ class name: pybind11::object

	Global dimension:			9
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [20, 20, 0.01, 0.005, 1, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

(array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([2.0e+01, 2.0e+01, 1.0e-02, 5.0e-03, 1.0e+00, 2.0e+00, 3.0e+00,
       6.0e+03, 1.5e+01]))


In [12]:
# Using CMA-ES
algo = pg.algorithm(pg.cmaes(gen = 20, sigma0=0.3))
print(algo)

Algorithm name: CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy [stochastic]
	C++ class name: pagmo::cmaes

	Thread safety: basic

Extra info:
	Generations: 50
	cc: auto
	cs: auto
	c1: auto
	cmu: auto
	sigma0: 0.3
	Stopping xtol: 1e-06
	Stopping ftol: 1e-06
	Memory: false
	Verbosity: 0
	Force bounds: false
	Seed: 354399482


In [13]:
archi = pg.archipelago(n=16, algo = algo, prob=prob, pop_size=50)
archi.evolve(); archi.wait()

[2.22074130e+00 8.08434369e+00 1.12060517e-03 3.76007606e-03
 6.23382177e-01 7.96437297e-01 2.52842935e+00 1.31754227e+03
 9.54183261e+00]
[0.         0.         0.         0.         5.         0.0935426
 7.44520514 0.30147958]
[1.46208707e+01 4.89395975e+00 1.96352997e-03 1.00337840e-03
 8.98769643e-01 2.31131174e-01 1.77243202e+00 2.96553238e+03
 1.19743925e+01]
[0.         0.         0.         0.         5.         0.34311825
 6.55282684 0.28287146]
[1.67640507e+01 1.79389479e+01 2.11249464e-04 3.86719401e-04
 9.43341806e-01 1.73832406e-01 1.85490903e+00 4.53686410e+03
 3.65103646e+00]
[ 0.          0.          0.          0.          5.         88.2965739
  6.18956617  3.11891834]
[6.22683337e+00 4.61378981e+00 3.33429719e-03 4.22535893e-03
 3.70603537e-01 1.98154968e+00 8.46142356e-01 4.60088833e+02
 1.15458057e+01]
[ 0.          0.          0.          0.          5.         19.57543155
  7.93962056  3.79089052]
[1.00553948e+01 1.82718995e+01 3.31857256e-04 4.82481511e-03
 3.45

[0.         0.         0.         0.         5.         0.10696224
 8.58826787 0.20303248]
[1.58510461e+01 1.56680281e+01 3.00532689e-03 4.31215990e-03
 5.17961873e-01 1.50156617e+00 2.35133629e+00 4.32454095e+03
 1.15126721e+01]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 4.00153844e-01 8.48426952e+00 5.68197484e-03]
[5.33482149e-01 5.82611164e+00 9.54039133e-03 3.10499605e-03
 9.97546382e-01 1.47096630e+00 1.57734267e+00 8.39157195e+01
 4.46444799e+00]
[0.         0.         0.         0.         5.         8.18871056
 1.22664586 5.36660258]
[1.71801928e+01 1.91973802e+01 2.18019231e-03 4.49648107e-03
 1.12015190e-01 4.56745073e-01 5.49906768e-01 3.69053419e+03
 1.06259026e+01]
[  0.           0.           0.           0.           5.
 703.51694765  25.42591706   4.72639883]
[1.20981020e+00 1.18506429e+01 7.18445439e-03 4.71575391e-03
 3.43972992e-01 9.34553620e-01 1.04353511e+00 1.94163321e+02
 7.57145972e+00]
[ 0.          0.          0.          0.

[0.         0.         0.         0.         5.         0.20038022
 7.08790991 0.25989568]
[3.98453859e+00 7.44579199e+00 4.29990644e-03 2.09283473e-03
 7.84617455e-01 1.65632270e+00 5.86479577e-01 1.89502904e+03
 1.17075735e+01]
[  0.           0.           0.           0.           5.
 116.77356272   4.74391502   4.933037  ]
[1.28171969e+01 1.81622097e+01 6.55873156e-03 4.73999423e-03
 9.08681602e-01 3.71954408e-01 1.95139403e+00 4.25352028e+03
 1.27266267e+01]
[0.         0.         0.         0.         5.         0.42000853
 6.44816222 0.0318449 ]
[9.21450373e+00 1.76210951e+01 3.57412618e-03 1.49879467e-03
 8.16288555e-01 1.83486431e+00 8.95553835e-01 3.22610565e+03
 3.31332475e+00]
[  0.           0.           0.           0.           5.
 563.81470479   3.76904569   6.78032547]
[1.49765805e+01 1.08269531e+00 1.23874544e-03 4.01127046e-03
 2.30399860e-01 3.94034179e-01 2.77132615e+00 7.92543835e+02
 9.63690382e+00]
[ 0.          0.          0.          0.          5.          9.

[  0.           0.           0.           0.           5.
 801.28343879   4.97530229   6.8353141 ]
[8.33133770e+00 1.68122703e+01 5.12609330e-03 2.15154129e-03
 1.60690743e-01 5.32831053e-01 1.19784537e-01 2.51341441e+03
 1.23060402e+01]
[  0.           0.           0.           0.           5.
 487.75084577   4.33786144  25.4677947 ]
[1.39645208e+01 1.80060229e+01 1.35913005e-03 1.62984455e-03
 1.04101024e-02 1.00794390e+00 1.41667690e+00 5.11760461e+03
 2.90859082e+00]
[   0.            0.            0.            0.            5.
 1000.71515305   69.27768663   37.37344821]
[1.78840891e+01 4.92577407e+00 8.30580232e-03 4.77967153e-03
 3.52085090e-02 2.07793427e-01 2.41539993e+00 5.16648466e+03
 1.04265125e+01]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.19900000e+00 1.58196130e+00 3.25791147e+01 4.07493630e-06]
[4.62468074e+00 1.50391356e+01 8.82848702e-03 1.70496680e-03
 8.60154457e-02 1.34913170e+00 1.36965501e+00 3.88465078e+03
 5.68100757e+00]
[ 0.          0.

[ 0.          0.          0.          0.          5.         48.2052752
  3.45740174  8.75245213]
[1.57390614e+01 1.35286139e+00 1.43363035e-03 1.21022457e-03
 5.70944426e-01 2.59510348e-03 2.30165411e+00 2.12832250e+03
 9.41066472e+00]
[0.         0.         0.         0.         5.         0.03856413
 8.35932407 1.39608764]
[1.07309878e+01 3.32161174e+00 3.88029058e-03 3.80245801e-03
 7.91610474e-01 1.14617703e+00 2.58675050e+00 6.89540619e+02
 1.54901611e-01]
[   0.            0.            0.            0.            5.
 1928.00477473    8.36665242   14.3253291 ]
[8.65787981e+00 1.65781397e+01 2.45008354e-03 4.46723431e-04
 9.74281904e-01 7.88421240e-01 1.78089867e+00 3.65398099e+03
 1.80451672e-01]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.89678680e+02 1.20437148e-01 2.19443261e+01]
[9.76811400e+00 1.66192485e+01 8.42432883e-03 1.76536184e-03
 3.87525864e-01 1.88726921e-01 2.63823212e+00 1.70206325e+03
 3.99930768e+00]
[0.         0.         0.

...

[0.         0.         0.         0.         5.         3.20564596
 7.52961331 0.01136093]
[4.53416647e+00 1.64588609e+01 9.33870335e-03 3.14770200e-03
 4.17763499e-01 1.35734879e+00 2.86084047e+00 2.51607607e+02
 6.78900519e+00]
[0.         0.         0.         0.         5.         0.28815984
 9.28508608 0.20566461]
[1.00838852e+01 1.36056479e+01 6.74315448e-03 1.83402583e-03
 8.92563389e-01 2.71562169e-01 2.41498011e+00 2.25096604e+02
 1.04474929e+01]
[0.         0.         0.         0.         5.         0.10795941
 5.92737797 1.47386078]
[1.53929066e+01 1.67548991e+01 5.44691001e-03 3.72380147e-03
 4.99548162e-01 1.89094978e+00 2.03657770e+00 3.30914982e+03
 1.39566566e+01]
[0.         0.         0.         0.         5.         0.39847241
 8.68211515 0.00979217]
[1.68605135e+01 1.08348129e+01 7.71535352e-03 4.50417197e-05
 9.29547864e-01 1.68528129e+00 1.24800417e+00 2.22086708e+03
 6.29529884e+00]
[ 0.          0.          0.          0.          5.         30.01549482
 11.154

[ 0.          0.          0.          0.          5.          0.04549436
 26.25446662  0.36825112]
[1.09754629e+01 1.69888525e+01 6.57573136e-03 9.62303281e-04
 1.77730506e-01 1.41751824e+00 8.44443888e-01 2.21229263e+03
 8.28289458e-01]
[  0.           0.           0.           0.           5.
 422.86104342   1.38676121  17.20425025]
[8.72068433e+00 1.42376930e+00 4.39533587e-03 1.82505593e-03
 3.48781617e-01 1.27972941e+00 7.24213170e-01 2.35093204e+02
 4.71558777e+00]
[ 0.          0.          0.          0.          5.         50.91358295
  3.55445983  2.84032929]
[5.51842379e+00 1.20155803e+01 1.88282292e-04 1.87303921e-03
 9.12487632e-01 1.77957261e+00 1.23497518e+00 3.22718391e+03
 1.16757664e+01]
[0.         0.         0.         0.         5.         0.0666922
 6.25407343 0.11511509]
[9.22334962e-01 2.89602983e+00 9.50433253e-03 2.15735075e-03
 1.90628526e-01 6.95307799e-01 2.24806832e+00 5.40244032e+03
 3.20930661e+00]
[  0.           0.           0.           0.           5.

[   0.            0.            0.            0.            5.
 1130.59485548   12.75571232   22.23385708]
[1.80470263e+00 6.14830407e+00 7.66180321e-03 3.06447645e-03
 7.33353534e-01 1.48754043e+00 1.54869927e+00 2.32934764e+03
 3.09520800e+00]
[   0.            0.            0.            0.            5.
 1907.80380395   14.51988508   16.55636889]
[1.01316396e+01 1.12875004e+01 1.50885577e-03 3.78531543e-03
 2.42319644e-01 1.28569321e+00 2.47467584e+00 4.33363530e+03
 4.75087920e+00]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 2.88567000e-01 1.23264599e+01 9.77043018e-03]
[1.95808424e+01 1.32028867e+01 5.90843770e-03 1.09812998e-03
 5.22716014e-01 1.79016310e+00 1.10020427e+00 5.81264008e+03
 1.48050769e+01]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.31630000e+00 2.82087071e-01 8.49128975e+00 2.84472388e-04]
[1.37839975e+01 1.81202778e+01 7.83954189e-03 2.69155468e-03
 4.80978312e-01 1.92528865e-01 2.40903922e+00 4.86794474e+03
 

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.72740000e+00 2.06626747e+00 7.23096697e+00 5.68043294e-05]
[1.75059304e+00 4.54536910e+00 7.23993992e-03 1.43359485e-03
 7.14649241e-01 5.48637899e-01 1.43292258e+00 4.40960892e+03
 5.00013992e+00]
[  0.           0.           0.           0.           5.
 189.16170725   5.54572089   2.35046529]
[1.30316267e+00 1.53919657e+01 9.56050991e-03 4.88588839e-03
 1.76605946e-01 1.82806775e+00 2.21103513e-01 5.75849729e+03
 8.21715826e+00]
[   0.            0.            0.            0.            5.
 1464.73921032   19.11908613    5.97946728]
[7.58687083e+00 7.19271899e-01 7.56564217e-03 2.66270816e-03
 5.39155080e-01 1.12746545e-01 6.35042343e-01 2.00607303e+03
 1.20039210e+01]
[ 0.          0.          0.          0.          5.         82.27372274
  5.45652626  3.35610611]
[4.99936106e+00 9.70190607e-01 9.40481359e-03 4.61815183e-03
 3.23004946e-01 1.72584774e+00 3.85171008e-01 5.64954912e+03
 8.76490852e+00]
[   0.          

[  0.           0.           0.           0.           5.
 439.90686482   0.94516774  30.64990837]
[1.73823233e+01 1.95758826e+01 9.48747608e-03 2.73176279e-03
 9.64529714e-01 8.47108269e-01 1.51475275e+00 5.91617355e+03
 1.08954362e+01]
[0.         0.         0.         0.         5.         0.43660899
 6.30641771 0.01199905]
[7.46819736e+00 1.45747085e+01 5.72872176e-03 4.36221993e-03
 2.87745392e-01 1.81362527e+00 2.83968770e+00 4.86544306e+02
 3.25118438e+00]
[ 0.          0.          0.          0.          5.         64.53393465
 10.47462221  2.88416916]
[3.26555506e+00 6.78725762e-01 3.75270502e-03 3.01832993e-03
 1.22829588e-01 7.99661197e-01 2.51757031e+00 3.33239230e+03
 1.24051959e+01]
[ 0.          0.          0.          0.          5.          1.32096406
 16.84543363  0.16635555]
[9.23713363e+00 1.57180951e+01 3.99495694e-03 6.54724138e-04
 1.34294137e-01 1.56290488e+00 1.82434333e+00 5.79646136e+03
 1.48237624e+00]
[  0.           0.           0.           0.           5

In [14]:
a = archi.get_champions_f()
a2 = sorted(archi.get_champions_f(), key = lambda x: x[0])[0]
best_isl_idx = [(el == a2).all() for el in a].index(True)
x_best = archi.get_champions_x()[best_isl_idx]
f_best = archi.get_champions_f()[best_isl_idx]
print("Best relaxed solution, x: {}".format(x_best)) 
print("Best relaxed solution, f: {}".format(f_best))

Best relaxed solution, x: [8.68414815e+00 5.15899056e+00 3.06411175e-03 5.44354454e-04
 9.73928506e-01 1.42401726e+00 2.25419798e+00 1.96513966e+03
 1.31109210e+01]
Best relaxed solution, f: [7.9891413]


In [ ]:
def gripper_forward_kinematics_example():
    urdf_path = './urdfs/DIVAR2V/urdf/DIVAR2V_eef.urdf'
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    # Transform for the robot location
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([-0.1, 0.5, 1.25]))
#     X_R = RigidTransform(RotationMatrix(RollPitchYaw([0.8, -0.8, 0])), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    # Spawn table
    table = Parser(plant, scene_graph).AddModelFromFile('./urdfs/table/extra_heavy_duty_table_modified.sdf','table')
    # Spawn spherical work piece
    sphere = Parser(plant, scene_graph).AddModelFromFile('./urdfs/helper/sphere.urdf','sphere')
    # Place the sphere at the center of the table
    # Length, width and height and thickness of the table
    # 1.39, 0.762, 0.736, 0.057
    # Sphere radius -- Can be made a design variable
    # 0.15
    # We can sample end-points on the surface of the sphere
    X_R = RigidTransform(RotationMatrix(RollPitchYaw([0, 0, 0])), np.array([0, 0, 0.736]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(sphere)[0]).body_frame(), X_R)

    plant.Finalize()
    
    end_frame = "eef"
    
    # Draw the frames
    for body_name in ["base_link", end_frame]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    plant.SetPositions(plant.GetMyContextFromRoot(context),
                  plant.GetModelInstanceByName("diva_robot"),
                  q0)
    
    gripper = plant.GetBodyByName(end_frame)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()